# Get data (use AOI of water_body app)
This notebook uses the **PySTAC Client** (```pystac-client```, https://pystac-client.readthedocs.io/en/latest/) Python package to find suitable Sentinel-2 MSIL2A data by using a query with a number of defined paramteres. 

The STAC API for AWS is available on the endpoint (https://earth-search.aws.element84.com/v0/collections/).

# Set-up

In [1]:
import pystac
from pystac_client import Client

from shapely.geometry import box, mapping, Polygon
import shapely.wkt

import os
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import xarray as xr
import stackstac

In [2]:
# Access to Catalog 
URL = "https://earth-search.aws.element84.com/v0/" # target STAC API for AWS

headers = []

cat = Client.open(URL, headers=headers, ignore_conformance=True)
cat

id: earth-search
title: Earth Search
description: A STAC API of AWS Public Datasets powered by stac-server
stac_api_version: 0.9.0
type: Catalog
id: sentinel-s2-l2a
title: Sentinel 2 L2A
"description: Sentinel-2a and Sentinel-2b imagery, processed to Level 2A (Surface Reflectance)"
providers: ESA (producer) Sinergise (processor) AWS (host) Element 84 (processor)
type: Collection
https://stac-extensions.github.io/item-assets/v1.0.0/schema.json


## Define params
A couple of S2 images (taken from the Water Bodies app) are:
- "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
- "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A

In [30]:
# Collection 
collections=["sentinel-s2-l2a-cogs"], 

# Start and End dates
start_date_str = '2021-07-12'; start_date = datetime.fromisoformat(start_date_str)
stop_date_str = '2021-07-14'; stop_date = datetime.fromisoformat(stop_date_str)

# BBOX or AOI
# bbox='-121.399,39.834,-120.74,40.472' #aoi = WKT format --> bbox=shapely.wkt.loads(aoi).bounds 
aoi='POLYGON((-121.73 40.52,-120.71 40.52,-120.71 39.79,-121.73 39.79,-121.73 40.52))'; bbox=shapely.wkt.loads(aoi).bounds 

# Other metadata
product_id = 'S2B_MSIL2A_20210713T184919_N0301_R113_T10TFK_20210713T213143'
cloud_cover = 30

# Define EPSG code
epsg = 'EPSG:4326'

## Query the Catalog

In [32]:
# # Query by Product ID
# query = cat.search(
#     collections=collections, 
#     sortby="properties.datetime",
#     ids = ['S2A_10TFK_20220524_0_L2A','S2B_10TFK_20210713_0_L2A']
# )

In [33]:
# Query by AOI, start and end date and other params 
query = cat.search(
    collections=collections, 
    datetime = (start_date, stop_date),
    sortby="properties.datetime",
    bbox=bbox, 
    # query = {
    #     'eo:cloud_cover': {'lt': 15}, # cloud_cover less than 15 percent by default
    #     # 'sentinel:product_id': {'eq': product_id}
    # }
)

## Create multi-dimensional array with ```stackstac```

In [34]:
# items_stack = query.get_all_items()
items = query.get_all_items()

print(f'There are {len(items)} items in the collection')
display(items)

There are 2 items in the collection


id: S2B_10TFK_20210713_0_L2A
"bbox: [-121.8343226741975, 39.63588071728383, -120.51956282559038, 40.64479052153662]"
datetime: 2021-07-13T19:03:24Z
platform: sentinel-2b
constellation: sentinel-2
instruments: ['msi']
gsd: 10
view:off_nadir: 0
proj:epsg: 32610
sentinel:utm_zone: 10
sentinel:latitude_band: T


In [7]:
# Display properties of one item 
items[0].properties

{'datetime': '2021-07-13T19:03:24Z',
 'platform': 'sentinel-2b',
 'constellation': 'sentinel-2',
 'instruments': ['msi'],
 'gsd': 10,
 'view:off_nadir': 0,
 'proj:epsg': 32610,
 'sentinel:utm_zone': 10,
 'sentinel:latitude_band': 'T',
 'sentinel:grid_square': 'FK',
 'sentinel:sequence': '0',
 'sentinel:product_id': 'S2B_MSIL2A_20210713T184919_N0301_R113_T10TFK_20210713T213143',
 'sentinel:data_coverage': 100,
 'eo:cloud_cover': 0,
 'sentinel:valid_cloud_cover': True,
 'created': '2021-07-13T23:57:53.846Z',
 'updated': '2021-07-13T23:57:53.846Z'}

**Note**: If the ```epsg``` and ```resolution``` are not defined in all Items/Assets, they must be explicitly defined in the ```stackstac.stack()``` call. The ```resolution``` refers to the output resolution and must be set in the same unit as the ```epsg``` field ([stackstac documentation](https://stackstac.readthedocs.io/en/latest/api/main/stackstac.stack.html)).

In [8]:
# Create Stack xarray
stack = stackstac.stack(items, 
                        # epsg=epsg_no, 
                        # resolution=10, #resolution=0.0000898 # this is in degrees (in case of epsg:4326 unprojected), corresponding to 10 meters)
                        # bounds_latlon=bbox, # filter by bounding box
                        sortby_date='asc'
                       )
stack

<xarray.DataArray 'stackstac-3eef8d21665eeacb4f8124a9bb94b313' (time: 2,
                                                                band: 17,
                                                                y: 10980,
                                                                x: 10980)>
dask.array<fetch_raster_window, shape=(2, 17, 10980, 10980), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/28)
  * time                          (time) datetime64[ns] 2021-07-13T19:03:24 2...
    id                            (time) <U24 'S2B_10TFK_20210713_0_L2A' 'S2A...
  * band                          (band) <U8 'overview' 'visual' ... 'WVP' 'SCL'
  * x                             (x) float64 6e+05 6e+05 ... 7.098e+05
  * y                             (y) float64 4.5e+06 4.5e+06 ... 4.39e+06
    sentinel:sequence             <U1 '0'
    ...                            ...
    sentinel:boa_offset_applied   (time) object None True
    title                         (band) <U31 'True color image' ... 'Scene C...
    common_name                   (band) object None None ... None None
    center_wavelength             (band) object None None 0.4439 ... None None
    full_width_half_max           (band) object None None 0.027 ... None None
    epsg                          int64 32610
Attributes:
    spec:        RasterSpec(epsg=32610, bounds=(600000.0, 4390200.0, 709800.0...
    crs:         epsg:32610
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 4500000.00|\n| 0.0...
    resolution:  10.0

In [10]:
# print id of the first item
index = 0
item_id = stack[index].id.data
print(f'id: {item_id}')
date = np.datetime_as_string(stack[index].id.time.values, unit="D")
print(f'date: {date}')
print(f'href: {item_id}')

# print available bands
print(f'bands: {stack[index].band.data}')
# print href of an asset/band
band = 'B04'
print(f'href of band "{band}": {items.items[index].get_assets()[band].href}')

id: S2B_10TFK_20210713_0_L2A
date: 2021-07-13
href: S2B_10TFK_20210713_0_L2A
bands: ['overview' 'visual' 'B01' 'B02' 'B03' 'B04' 'B05' 'B06' 'B07' 'B08' 'B8A'
 'B09' 'B11' 'B12' 'AOT' 'WVP' 'SCL']
href of band "B04": https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B04.tif


In [11]:
print(f'shape: {stack[0].sel(band=band).shape}')
print(f'Show some values of the array:')
display(stack[0].sel(band=band).values[0:10,0:10])

shape: (10980, 10980)
Show some values of the array:


array([[304., 579., 534., 373., 501., 426., 300., 224., 326., 442.],
       [417., 596., 633., 352., 362., 346., 350., 231., 336., 490.],
       [604., 520., 539., 483., 464., 448., 384., 330., 373., 459.],
       [388., 343., 484., 552., 581., 457., 363., 382., 474., 583.],
       [631., 769., 644., 644., 655., 729., 546., 424., 584., 715.],
       [564., 760., 725., 615., 561., 836., 489., 450., 518., 557.],
       [484., 686., 645., 466., 405., 479., 441., 416., 341., 505.],
       [483., 686., 685., 474., 442., 554., 617., 471., 359., 392.],
       [566., 665., 655., 682., 587., 526., 539., 678., 548., 500.],
       [466., 476., 465., 683., 663., 368., 476., 778., 670., 623.]])

## Plotting

In [ ]:
# Plot a single band 
plt.figure()
plt.imshow(stack[0].sel(band=band))#,norm=colors.Normalize(0, 4000))
plt.title(f'{date}, {band}')